<a href="https://colab.research.google.com/github/xiaoyufan/nbme/blob/main/baseline_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBME Baseline DeBERTa

## Configurations

In [1]:
class Config:
  batch_size = 16
  device = 'tpu'
  epochs = 1
  input_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Preprocessed'
  mode = 'dev'
  model = 'microsoft/deberta-base'
  output_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output'
  sequence_max_length = 466

## Packages

In [2]:
!pip install transformers
!pip install tokenizers

!pip install --force-reinstall git+https://github.com/xiaoyufan/nbme.git

!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

  Cloning https://github.com/xiaoyufan/nbme.git to /tmp/pip-req-build-59zhv7b3
  Running command git clone -q https://github.com/xiaoyufan/nbme.git /tmp/pip-req-build-59zhv7b3
  Created wheel for nbme: filename=nbme-1.0-py3-none-any.whl size=2308 sha256=1447d17552c3e0eb4e9c616cd51ff9b8714a1fa73fd89faf55560d1927c473f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-jgqau6pl/wheels/e2/dc/dd/b061e30220e414c0a096e4fbaddd405c1b526d1453d9a447e3
Successfully built nbme
  Attempting uninstall: nbme
    Found existing installation: nbme 1.0
    Uninstalling nbme-1.0:
      Successfully uninstalled nbme-1.0
  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl (152.9 MB)


In [3]:
import pandas as pd
import torch
import torch.nn as nn

from nbme_utils import scoring
from transformers import AutoConfig, AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, Dataset

## Device

In [ ]:
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if Config.device == 'tpu':
  assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

  import torch_xla
  import torch_xla.core.xla_model as xm

  device = xm.xla_device()

device

## Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(Config.model)
tokenizer.save_pretrained(f'{Config.output_dir}/tokenizer')

## Data Loading

### Defining Dataset

In [ ]:
train = pd.read_csv(f'{Config.input_dir}/train.csv')
valid = pd.read_csv(f'{Config.input_dir}/validate.csv')
train.shape, valid.shape

In [ ]:
if Config.mode == 'dev':
  train = train.sample(n=800, random_state=0).reset_index(drop=True)
  valid = valid.sample(n=200, random_state=0).reset_index(drop=True)
train.shape, valid.shape

In [ ]:
train.head()

In [ ]:
def generate_labels(encoded):
  labels = torch.zeros(len(encoded['input_ids']))

  for idx, (seq_id, offsets) in enumerate(zip(encoded["sequence_ids"], encoded["offset_mapping"])):
    if not seq_id:
      labels[idx] = -1
      continue

    

  return labels

class NBMEDataset(Dataset):
  def __init__(self, data: pd.DataFrame, tokenizer: AutoTokenizer, config: Config):
    self.data = data
    self.tokenizer = tokenizer
    self.config = config

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx: int):
    sample = self.data.iloc[idx]

    encoded = self.tokenizer(
      sample['pn_history'],
      sample['feature_text'],
      # TODO: Compute max length of sequences
      max_length=self.config.sequence_max_length,
      padding='max_length',
      return_offsets_mapping=True,
    )

    y_true = generate_labels(encoded)

    x = encoded.copy()
    del x['offset_mapping']
    x = {k: torch.tensor(v, dtype=torch.long) for k, v in x.items()}

    return x, y_true

In [ ]:
train_dataset = NBMEDataset(train, tokenizer, Config)
valid_dataset = NBMEDataset(valid, tokenizer, Config)

### Dataset Loader

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=Config.batch_size,
    shuffle=True,)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=Config.batch_size,
    shuffle=False,)

In [ ]:
x, y_true = next(iter(train_loader))

## Model

In [ ]:
class NBMEDebertaBaseline(nn.Module):
  def __init__(self):
    super().__init__()

    config = AutoConfig.from_pretrained(Config.model, output_hidden_states=True)
    self.model = AutoModel.from_pretrained(Config.model, config=config)

    self.fc = nn.Linear(config.hidden_size, 1)

  def forward(self, data):
    last_out = self.model(**data)[0]
    logits = self.fc(last_out)
    return logits

In [ ]:
model = NBMEDebertaBaseline()

## Training

## Inference